<a href="https://colab.research.google.com/github/TakaakiFukunaga/graduate-school/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path 

path = Path('summarization')
if not path.exists():
    path.mkdir(parents=True)

In [ ]:
!pip install -qU torch==1.13.* torchtext==0.14.* torchvision==0.14.* torchaudio==0.13.* torchmetrics==0.11.* \
    transformers==4.26.1 pytorch_lightning==1.9.3 sentencepiece==0.1.97

In [ ]:
!pip install neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import math
import time
import copy
from tqdm import tqdm
import pandas as pd
import tarfile
import neologdn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [ ]:
class Settings:
  def __init__(self):



    self.MODEL_NAME = "sonoisa/t5-base-japanese"
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    self.max_length_src = 400
    self.max_length_target = 200

    self.batch_size_train = 8
    self.batch_size_valid = 8
    
    self.epochs = 1000
    self.patience = 20

settings = Settings()

In [ ]:
body_data = pd.read_csv('body_data.csv')
summary_data = pd.read_csv('summary_data.csv')

pd.merge(
    body_data.query('text.notnull()', engine='python').rename(columns={'text': 'body'}),
    summary_data.rename(columns={'text': 'summary'}),
    on='article_id', how='inner'
).sort_values('article_id').head(10)

,article_id,title,body,summary
3336,10340479,有名企業の役員も御用達！ 接待を成功に導くレストラン５選,\n仕事ができる人は使えるレストランを知っている。いま注目の企業の経営者が、ここぞというとき...,有名企業の役員御用達の接待を成功に導くレストラン5選を紹介している
3338,10340479,有名企業の役員も御用達！ 接待を成功に導くレストラン５選,\n仕事ができる人は使えるレストランを知っている。いま注目の企業の経営者が、ここぞというとき...,「IL BUTTERO」「フレンチ キッチン」
3337,10340479,有名企業の役員も御用達！ 接待を成功に導くレストラン５選,\n仕事ができる人は使えるレストランを知っている。いま注目の企業の経営者が、ここぞというとき...,「料亭 三長」「分とく山」「居酒屋 東京十月」
8876,10422202,子どもの友だち、家に上げる？上げない？,\n子どもがお友達と遊ぶのが楽しくなってくる年齢になると、子どもに「お友達とうちで遊びたい」...,「よそのお宅にも上がらないように言っている」との声もあがった
8875,10422202,子どもの友だち、家に上げる？上げない？,\n子どもがお友達と遊ぶのが楽しくなってくる年齢になると、子どもに「お友達とうちで遊びたい」...,ある母親は、押し入れやタンスを開けられて以来、出入り禁止にしたとのこと
8874,10422202,子どもの友だち、家に上げる？上げない？,\n子どもがお友達と遊ぶのが楽しくなってくる年齢になると、子どもに「お友達とうちで遊びたい」...,子供の友達を「家に入れない」主義の親がいるという
10991,10428883,あなたは大丈夫？不倫にハマるのはこんな人,\n恋の悩みは尽きないもの。そして、恋の悩みの中でも、より一層泥沼化しやすい悩みが『不倫』で...,寂しがり屋、普段あまり異性との交流がないの5つ
10990,10428883,あなたは大丈夫？不倫にハマるのはこんな人,\n恋の悩みは尽きないもの。そして、恋の悩みの中でも、より一層泥沼化しやすい悩みが『不倫』で...,雰囲気に流されやすい、NOと言うのが苦手、何歳になっても恋バナが好き
10989,10428883,あなたは大丈夫？不倫にハマるのはこんな人,\n恋の悩みは尽きないもの。そして、恋の悩みの中でも、より一層泥沼化しやすい悩みが『不倫』で...,不倫にハマりやすい人の傾向5つを紹介している
12845,10456562,情熱的な恋＆純愛を楽しめる星座相性ランキング,\n1情熱的な恋愛になる星座ランキング〜心が熱く燃え上がる星の組み合わせは？〜第１位「さそり...,情熱的な恋愛になる星座の1位は、「さそり座⇔ふたご座」となった


In [ ]:
def join_text(x, add_char='。'):
    return add_char.join(x)

def preprocess_text(text):
    text = re.sub(r'[\r\t\n\u3000]', '', text)
    text = neologdn.normalize(text)
    text = text.lower()
    text = text.strip()
    return text

summary_data = summary_data.query('text.notnull()', engine='python').groupby(
    'article_id'
).agg({'text': join_text})

body_data = body_data.query('text.notnull()', engine='python')

data = pd.merge(
    body_data.rename(columns={'text': 'body_text'}),
    summary_data.rename(columns={'text': 'summary_text'}),
    on='article_id', how='inner'
).assign(
    body_text=lambda x: x.body_text.map(lambda y: preprocess_text(y)),
    summary_text=lambda x: x.summary_text.map(lambda y: preprocess_text(y))
)

In [ ]:
def convert_batch_data(train_data, valid_data, tokenizer):

    def generate_batch(data):

        batch_src, batch_tgt = [], []
        for src, tgt in data:
            batch_src.append(src)
            batch_tgt.append(tgt)

        batch_src = tokenizer(
            batch_src, max_length=settings.max_length_src, truncation=True, padding="max_length", return_tensors="pt"
        )
        batch_tgt = tokenizer(
            batch_tgt, max_length=settings.max_length_target, truncation=True, padding="max_length", return_tensors="pt"
        )

        return batch_src, batch_tgt

    train_iter = DataLoader(train_data, batch_size=settings.batch_size_train, shuffle=True, collate_fn=generate_batch)
    valid_iter = DataLoader(valid_data, batch_size=settings.batch_size_valid, shuffle=True, collate_fn=generate_batch)

    return train_iter, valid_iter

tokenizer = T5Tokenizer.from_pretrained(settings.MODEL_NAME, is_fast=True)

X_train, X_test, y_train, y_test = train_test_split(
    data['body_text'], data['summary_text'], test_size=0.2, random_state=42, shuffle=True
)

train_data = [(src, tgt) for src, tgt in zip(X_train, y_train)]
valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]

train_iter, valid_iter = convert_batch_data(train_data, valid_data, tokenizer)

In [ ]:
class T5FineTuner(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.model = T5ForConditionalGeneration.from_pretrained(settings.MODEL_NAME)

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None,
        decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

In [ ]:
def train(model, data, optimizer, PAD_IDX):
    
    model.train()
    
    loop = 1
    losses = 0
    pbar = tqdm(data)
    for src, tgt in pbar:
                
        optimizer.zero_grad()
        
        labels = tgt['input_ids'].to(settings.device)
        labels[labels[:, :] == PAD_IDX] = -100

        outputs = model(
            input_ids=src['input_ids'].to(settings.device),
            attention_mask=src['attention_mask'].to(settings.device),
            decoder_attention_mask=tgt['attention_mask'].to(settings.device),
            labels=labels
        )
        loss = outputs['loss']

        loss.backward()
        optimizer.step()
        losses += loss.item()
        
        pbar.set_postfix(loss=losses / loop)
        loop += 1
        
    return losses / len(data)

def evaluate(model, data, PAD_IDX):
    
    model.eval()
    losses = 0
    with torch.no_grad():
        for src, tgt in data:

            labels = tgt['input_ids'].to(settings.device)
            labels[labels[:, :] == PAD_IDX] = -100

            outputs = model(
                input_ids=src['input_ids'].to(settings.device),
                attention_mask=src['attention_mask'].to(settings.device),
                decoder_attention_mask=tgt['attention_mask'].to(settings.device),
                labels=labels
            )
            loss = outputs['loss']
            losses += loss.item()
        
    return losses / len(data)

In [ ]:
model = T5FineTuner()
model = model.to(settings.device)

optimizer = optim.Adam(model.parameters())

PAD_IDX = tokenizer.pad_token_id
best_loss = float('Inf')
best_model = None
counter = 1

for loop in range(1, settings.epochs + 1):

    start_time = time.time()

    loss_train = train(model=model, data=train_iter, optimizer=optimizer, PAD_IDX=PAD_IDX)

    elapsed_time = time.time() - start_time

    loss_valid = evaluate(model=model, data=valid_iter, PAD_IDX=PAD_IDX)

    print('[{}/{}] train loss: {:.4f}, valid loss: {:.4f} [{}{:.0f}s] counter: {} {}'.format(
        loop, settings.epochs, loss_train, loss_valid,
        str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
        elapsed_time % 60,
        counter,
        '**' if best_loss > loss_valid else ''
    ))

    if best_loss > loss_valid:
        best_loss = loss_valid
        best_model = copy.deepcopy(model)
        counter = 1
    else:
        if counter > settings.patience:
            break

        counter += 1

100%|██████████| 469/469 [11:07<00:00,  1.42s/it, loss=2.1]


[1/1000] train loss: 2.1027, valid loss: 1.8585 8s] counter: 1 **


100%|██████████| 469/469 [11:05<00:00,  1.42s/it, loss=1.54]


[2/1000] train loss: 1.5381, valid loss: 1.8866 5s] counter: 1 


100%|██████████| 469/469 [11:14<00:00,  1.44s/it, loss=1.2]


...

[9/1000] train loss: 0.4080, valid loss: 2.9436 19s] counter: 8 


100%|██████████| 469/469 [11:13<00:00,  1.44s/it, loss=0.326]


[10/1000] train loss: 0.3256, valid loss: 2.9992 14s] counter: 9 


 41%|████      | 191/469 [04:32<06:36,  1.42s/it, loss=0.247]


KeyboardInterrupt: ignored

In [ ]:
 # 保存
save_path = "my_model_training_state.pt"
torch.save({'loop': loop,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss_valid': loss_valid,
            'loss_train': loss_train},
           save_path)

In [ ]:

#　読み込み
model = TheModelClass(*args, **kwargs)
optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

PATH = "my_model_training_state.pt"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# optimizerのstateを現在のdeviceに移す。これをしないと、保存前後でdeviceの不整合が起こる可能性がある。
for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)
epoch = checkpoint['epoch']
loss = checkpoint['loss']

# model.eval()
# # - or -
model.train()

model = model.to(device)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
model_dir_path = Path('model')
if not model_dir_path.exists():
    model_dir_path.mkdir(parents=True)

tokenizer.save_pretrained(model_dir_path)
best_model.model.save_pretrained(model_dir_path)

In [ ]:
def generate_text_from_model(text, trained_model, tokenizer, num_return_sequences=1):

    trained_model.eval()
    
    text = preprocess_text(text)
    batch = tokenizer(
        [text], max_length=settings.max_length_src, truncation=True, padding="longest", return_tensors="pt"
    )

    # 生成処理を行う
    outputs = trained_model.generate(
        input_ids=batch['input_ids'].to(settings.device),
        attention_mask=batch['attention_mask'].to(settings.device),
        max_length=settings.max_length_target,
        repetition_penalty=8.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        # temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
        # num_beams=10,  # ビームサーチの探索幅
        # diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        # num_beam_groups=10,  # ビームサーチのグループ
        num_return_sequences=num_return_sequences,  # 生成する文の数
    )

    generated_texts = [
        tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for ids in outputs
    ]

    return generated_texts

tokenizer = T5Tokenizer.from_pretrained(model_dir_path)
trained_model = T5ForConditionalGeneration.from_pretrained(model_dir_path)
trained_model = trained_model.to(settings.device)

In [ ]:
index = 1
body = valid_data[index][0]
summaries = valid_data[index][1]
generated_texts = generate_text_from_model(
    text=body, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ 生成本文')
print('\n'.join(generated_texts[0].split('。')))
print()
print('□ 教師データ要約')
print('\n'.join(summaries.split('。')))
print()
print('□ 本文')
print(body)

□ 生成本文
リヴァプールとのテストマッチで、4-0と快勝を抑えたマインツ
昨季のelファイナリストから4得点をあげることは並大抵のことではないという
「結果については慎重に判断していかないと」と語った

□ 教師データ要約
マインツのシュミット監督が4-0で勝利したリバプール戦について語った
ポカール1回戦に向け「精力的にプレーしなくては勝利などできない」と指摘
ヨーロッパリーグに向けても「追い風となる」よう願った

□ 本文
週末に行われたリヴァプールとのテストマッチでは、非常に安定したタイトなディフェンスを披露し、4-0と快勝を抑えたマインツ。しかしマネージャーのシュレーダー氏は「結果については慎重に判断していかないと」と試合後に語った。 しかし昨季のelファイナリストから4得点をあげることは並大抵のことではないことは確かだ。オフェンス面っでもマインツは、サイドからのテンポと正確なパス、そしてその決定力でも納得のプレーを見せており、クレメンスは「めったにリヴァプールとは試合できないし、楽しかったよ」とコメント。この日主将を務めたベルは、ホームの大観衆の背を受けて「これまでのテストマッチよりはやりやすかったね」と振り返っている。 一方のシュミット監督は「ノバーラ戦でも同じように精力的にプレーしていれば敗戦しなかった」との見方を示し、2週間後に控えたポカール1回戦に向けて「同様に精力的にプレーしなくては勝利などできない」と述べ、また「貴重な経験」となるヨーロッパリーグに向けても「追い風となる」よう願った。
